# LlamaIndex Multi-Agent Catering System with MCP Server Integration

### Project Overview
An advanced multi-agent system built with LlamaIndex that automates catering menu planning (taking into guests' dietary requirements and chef availability) by orchestrating multiple MCP (Model Context Protocol) servers.

### MCP Server Architecture

1. **Catering Management Server** (`uv run catering_server.py`)
   - SQLite-based recipe and chef database
   - Core functionality:
     - Dietary analysis and requirements processing
     - Recipe and chef database management
     - Menu planning and chef matching
   - Tools:
     - `dietary_strategy_analyzer`: Smart analysis of group dietary needs
     - `get_safe_recipes_and_chefs`: Recipe-chef matching with dietary compliance
     - `list_all_specializations`: Chef specialization management
     - `get_chefs_by_specialization`: Specialized chef lookup


2. Filesystem Server (`npx @modelcontextprotocol/server-filesystem`)
- Handles result persistence
- Saves final recommendations to files

### Web Search Integration
- Integrates with Tavily API for recipe research
- Used when existing recipes don't meet requirements

### Multi-Agent Workflow
1. **Diet Analysis Agent**: Processes guest requirements and determines universal/alternative needs
2. **Recipe Finding Agent**: Queries database for matching recipes
3. **Research Agent**: Searches web for new recipes when needed
4. **Chef Matching Agent**: Pairs recipes with qualified chefs
5. **Filesystem Agent**: Saves final recommendations

### Key Features
- Analyzes group dietary restrictions and allergies
- Finds safe recipes matching all requirements
- Automatically researches alternatives when needed
- Matches recipes with specialized chefs
- Saves recommendations for reference

### Tech Stack
- LlamaIndex + OpenAI GPT-4o
- SQLite + SQLAlchemy
- Model Context Protocol (MCP)
- Tavily API
- Pydantic

## Load tools from MCP servers

In [1]:
# %pip install -qU llama-index-tools-mcp

In [2]:
import asyncio
from llama_index.tools.mcp import McpToolSpec
from llama_index.llms.openai import OpenAI
from llama_index.tools.mcp import BasicMCPClient
import os 
import dotenv

dotenv.load_dotenv()
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))


async def setup_tools():
    # RAG server for guests and recipes
    catering_client = BasicMCPClient("uv", args=["run", "catering_server.py"])
    
    # Fetch server for web search
    fetch_client = BasicMCPClient("uvx", args=["mcp-server-fetch"])
    
    file_system_client = BasicMCPClient("npx",
            args= [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "/Users/dangphuonganh/Documents/mcp/hackathon" #allowed directory
            ])
    # Create tool specs
    catering_tools = McpToolSpec(client=catering_client, 
                            allowed_tools=['dietary_strategy_analyzer','get_all_recipes', 'list_all_specializations', 'get_safe_recipes_and_chefs', 'get_chefs_by_specialization'], 
                            include_resources=True)
    
    fetch_tools = McpToolSpec(client=fetch_client, include_resources=False)
    
    filesystem_tools = McpToolSpec(
        client=file_system_client,
        allowed_tools=['list_allowed_directories', 'read_file', 'write_file', 'create_directory'],
        include_resources=False
    )
    # Get all tools
    catering_function_tools = await catering_tools.to_tool_list_async()
    fetch_function_tools = await fetch_tools.to_tool_list_async()
    filesystem_function_tools = await filesystem_tools.to_tool_list_async()
    return catering_function_tools, fetch_function_tools, filesystem_function_tools

# Get tools (run this once)
catering_tools, fetch_tools, filesystem_tools = await setup_tools()

# Filter tools by name
dietary_tools = [tool for tool in catering_tools if 'dietary_strategy_analyzer' in tool.metadata.name]
find_existing_recipe_tools = [tool for tool in catering_tools if 'get_safe_recipes_and_chefs' in tool.metadata.name]
list_all_specializations_tool = [tool for tool in catering_tools if 'list_all_specializations' in tool.metadata.name]
get_chefs_by_specialization_tool = [tool for tool in catering_tools if 'get_chefs_by_specialization' in tool.metadata.name]

for i, tool in enumerate(list_all_specializations_tool):
    print(f"Tool {i+1}: {tool.metadata.name}\n{tool.metadata.description}")


Tool 1: list_all_specializations

List all specializations in the database.
Return a set of specializations.



In [3]:
for i, tool in enumerate(dietary_tools):
    print(f"Tool {i+1}: {tool.metadata.name}\n{tool.metadata.description}")


Tool 1: dietary_strategy_analyzer

Analyzes guest dietary restrictions and determines menu strategy.

This function processes a list of guests and their dietary restrictions to determine:
1. Universal requirements (restrictions affecting 30%+ of guests)
2. Needed alternatives for minority restrictions
3. Total guest count

Args:
    guest_list (List[Dict[str, Any]]): List of guest dictionaries containing dietary requirements.
        Each guest dict can have any of these fields (all optional with default False):
        - is_vegan: bool
        - is_vegetarian: bool
        - is_gluten_free: bool
        - is_dairy_free: bool
        - allergens: List[str] (defaults to empty list)

Call it like this: dietary_strategy_analyzer(guest_list={input_guest_list})



## Search web tool

In [4]:
# %pip install -qU tavily-python

In [5]:
from tavily import AsyncTavilyClient

# search web tool
tavily_api_key = os.getenv("TAVILY_API_KEY")
# note the type annotations for the incoming query and the return string
async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))

## Define DietaryAnalysisOutput

In [6]:
from pydantic import BaseModel, Field
from typing import Dict, Any, List, Optional

class UniversalRequirement(BaseModel): 
    dietary_restrictions: List[str]  
    allergens: List[str] = Field(default_factory=list)

    class Config:
        extra = "forbid"

class AlternativeRequirement(BaseModel):
    dietary_restrictions: List[str] 
    allergens: List[str] = Field(default_factory=list)
    quantity_needed: int

    class Config:
        extra = "forbid"

class DietaryAnalysisOutput(BaseModel):
    total_guests: int
    universal_requirement: UniversalRequirement 
    alternatives_needed: List[AlternativeRequirement]
    
    class Config:
        extra = "forbid"

## Define FunctionAgent

In [12]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI
import os
import dotenv
dotenv.load_dotenv()
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

diet_analyze_agent = FunctionAgent(
    name="DietAnalyzeAgent",
    description="Comprehensively analyze guest dietary restrictions and allergies to determine optimal menu requirements.",
    system_prompt=(
    """You are an agent that analyzes guest dietary restrictions and allergies.

    You will be given only ONE guest list, containing dictionaries with dietary information for each guest.

    Simply pass this input directly to the dietary_strategy_analyzer tool
    
    Return the tool's output as it is. DO NOT modify it."""
    ),
    llm=llm,
    verbose=True,
    output_cls=DietaryAnalysisOutput,
    tools= dietary_tools 
)

find_existing_recipes_agent = FunctionAgent(
    name="FindExistingRecipeAgent",
    description="Find existing recipes and matching chefs for the given menu requirements based on dietary analysis.",
    system_prompt="""## You are an expert in finding recipes that meet the dietary requirements and match them with the best available chefs.
# Input Processing: 
## You will be given a requirement (Dict) containing 2 fields: dietary_restrictions and allergens

## Analysis Guideline:
Simply pass this input directly to the *get_safe_recipes_and_chefs* tool and return the tool's output as it is without modification

If no recipes are found, just output 'Failed'. Nothing else.
""",
    tools=find_existing_recipe_tools
)

chef_matching_agent = FunctionAgent(
    name="ChefMatchingAgent",
    description="Given a specialization, find matching chef in the database.",
    system_prompt=(
        """You are an agent that matches recipes with chefs based on 'specialization'
        1. you must start by using the list_all_specializations tool to get the list of existing chefs' specializations.
        2. Input processing: You will be given a recipe, you must categorize the recipe as one of the found specialization
        3. Finally, use the get_chefs_by_specialization tool to find chefs with that specialization"""),
    tools=[tool for tool in catering_tools if 'get_chefs_by_specialization' in tool.metadata.name] + list_all_specializations_tool,
)

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Create recipe(s) that satify dietary requirements",
    system_prompt="""You are an agent that finds specific recipe(s) that satify dietary requirements.
    
    # Input: You will be given a requirement (Dict) containing 2 fields: dietary_restrictions and allergens
    
    # You have the following tasks:
    Create new recipe(s) that satisfies the {{dietary_restrictions}} and contains none of these allergens {{allergens}} from the input
    Use can use the **search_web** tool to get ideas for the recipe.

    ## Output the recipe's details, including:
    - recipe_name
    - short_description
    - ingredients: List of ingredients needed
    
    ## Important: Valid recipe(s) criteria:
    1. The recipes' names must include specific ingredients e.g. 'Fresh mango salad' or 'Frech Chicken stew with lentils' not just a collection of recipes like 'Dairy-Free, Gluten-Free and Nut-Free Recipes'"
    2.  you must be able to specify the ingredients list e.g '3 ripe mangos, diced, 1 medium red bell pepper, chopped..'
    
    
    Do not ask for more questions at the end!
    """,
    tools= [search_web],
    verbose=True,
)

filesystem_agent = FunctionAgent(
    name="File System Agent",
    description="Agent using file system tools",
    tools=filesystem_tools,
    system_prompt="""You are an AI assistant for Tool Calling having access to the file system""",
)


### Test structured agent output

In [8]:
guest = [
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "is_dairy_free": True,
    "allergens": ["chicken"]
  },
  {
    "allergens": ['egg', 'soy']
    }
]  


from llama_index.core.agent.workflow import AgentWorkflow
response = await diet_analyze_agent.run(f"Analyze the guests: {guest}, just directly put the input as argument to the tool")
print("response", response)

print(response.structured_response)
print(response.get_pydantic_model(DietaryAnalysisOutput))

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
response {
  "total_guests": 7,
  "universal_requirement": {
    "dietary_restrictions": [
      "is_vegan",
      "is_vegetarian"
    ],
    "allergens": [
      "nuts"
    ]
  },
  "alternatives_needed": [
    {
      "dietary_restrictions": [
        "is_dairy_free"
      ],
      "allergens": [
        "chicken"
      ],
      "quant

In [9]:
analysis = response.structured_response
# Create requirements from both universal and alternative requirements
requirements = []

# Add universal requirement
if "universal_requirement" in analysis:
    requirements.append({
        'dietary_restrictions': analysis["universal_requirement"]["dietary_restrictions"],
        'allergens': analysis["universal_requirement"]["allergens"]
    })
    
# Add alternative requirements
if "alternatives_needed" in analysis:
    for alt in analysis["alternatives_needed"]:
        requirements.append({
            'dietary_restrictions': alt["dietary_restrictions"],
            'allergens': alt["allergens"]
        })

print("Requirements:", requirements)

Requirements: [{'dietary_restrictions': ['is_vegan', 'is_vegetarian'], 'allergens': ['nuts']}, {'dietary_restrictions': ['is_dairy_free'], 'allergens': ['chicken']}, {'dietary_restrictions': [], 'allergens': ['egg', 'soy']}]


## Main Workflow

In [13]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)

class DietaryAnalysisEvent(Event):
    guest_list: list

class FindExistingRecipeEvent(Event):
    requirement: dict

class SearchRecipeEvent(Event):
    requirement: dict

class MatchChefEvent(Event):
    recipes_found: str

class ReviewEvent(Event):
    requirement: dict
    search_result: str

class FinalizeEvent(Event):
    result: str
   

class CateringMultiAgentFlow(Workflow):
    retry_step1 = 0
    llm = llm

    @step
    async def setup(self, ctx: Context, ev: StartEvent) -> DietaryAnalysisEvent:
        await ctx.store.set("search_result", False)
        self.diet_analyze_agent = ev.diet_analyze_agent
        self.find_existing_recipes_agent = ev.find_existing_recipe_agent
        self.chef_matching_agent = ev.chef_matching_agent
        self.research_agent = ev.research_agent
        self.filesystem_agent = ev.filesystem_agent
        return DietaryAnalysisEvent(guest_list=ev.guest_list)

    @step
    async def diet_analyze(self, ctx: Context, ev: DietaryAnalysisEvent) -> FindExistingRecipeEvent | StopEvent:
        guest_list = ev.guest_list
        response = await self.diet_analyze_agent.run(f"Analyze the guests: {guest_list}, just directly put the input as argument to the tool")

        if not response:
            return StopEvent(result="No response from diet_analyze_agent")
        if not response.structured_response:
            return StopEvent(result="No structured response from diet_analyze_agent")
        
        try:
            analysis = response.structured_response
            # Create requirements from both universal and alternative requirements
            requirements = []

            # Add universal requirement
            if "universal_requirement" in analysis:
                requirements.append({
                    'dietary_restrictions': analysis["universal_requirement"]["dietary_restrictions"],
                    'allergens': analysis["universal_requirement"]["allergens"]
                })
                
            # Add alternative requirements
            if "alternatives_needed" in analysis:
                for alt in analysis["alternatives_needed"]:
                    requirements.append({
                        'dietary_restrictions': alt["dietary_restrictions"],
                        'allergens': alt["allergens"]
                    })

            print("Requirements:", requirements)
            
            if not requirements:
                return StopEvent(result="No requirements generated in diet_analyze")
            
            await ctx.store.set("requirements count", len(requirements))
            for requirement in requirements:
                ctx.send_event(FindExistingRecipeEvent(requirement=requirement))
            
        except Exception as e:
            print("Error", e)
            return StopEvent(result="Failed in diet_analyze")
        
    @step
    async def find_existing_recipes(self, ctx: Context, ev: FindExistingRecipeEvent) -> SearchRecipeEvent | FinalizeEvent:
        print("=== find_existing_recipes ===")
        diet_requirements = ev.requirement
        result = await self.find_existing_recipes_agent.run(f"Find the best recipes and matching chefs for the given requirements: {diet_requirements}")
        print("find existing recipes result", str(result))
        if "failed" in str(result).lower():
            # delegate to the research agent
            return SearchRecipeEvent(requirement=diet_requirements)
        else:
            return FinalizeEvent(result=str(result))

    @step 
    async def search_new_recipes(self, ctx: Context, ev: SearchRecipeEvent) -> ReviewEvent:
        print("=== search_new_recipes ===")
        diet_requirements = ev.requirement
        result = await self.research_agent.run(f"Search for new recipes online that match the given requirements: {diet_requirements}. Make sure to match the recipe's specialization with one of the existing chefs' specializations found from the list_all_specializations tool")
        print("search_new_recipes result", str(result))
        return ReviewEvent(search_result=str(result), requirement=diet_requirements)
    
    @step
    async def review_search_result(self, ctx: Context, ev: ReviewEvent) -> SearchRecipeEvent | MatchChefEvent:
        requirement = ev.requirement
        search_result = ev.search_result
        review_result = self.llm.complete(
            f"Review if the following recipes meet the guests' dietary requirements: \n\n Dietary requirements: {requirement} \n\n Recipes: {search_result}"
            "If no, output 'Failed'"
            "If yes, output 'Success'"
            "Plus justification for your answer"
            )
        print("review_result", review_result)
        # If not, search again
        if "failed" in str(review_result).lower():
            return SearchRecipeEvent(requirement=requirement + "feedback for previous result: " + str(review_result))
        else:
            return MatchChefEvent(recipes_found=str(search_result))        

    @step
    async def match_chef(self, ctx: Context, ev: MatchChefEvent) -> FinalizeEvent:
        result = await self.chef_matching_agent.run(f"Match the recipes with chefs that have the same specializations in the following list: {ev.recipes_found}")
        print("matching chef result", str(result))
        return FinalizeEvent(result=f'recipes: {ev.recipes_found}\n\nchefs: {str(result)}')
    
    @step
    async def finalize(self, ctx: Context, ev: FinalizeEvent) -> StopEvent:
        requirements_count = await ctx.store.get("requirements count")
        data = ctx.collect_events(ev, [FinalizeEvent] * requirements_count)
        if data is None:
            print("Not all requirements are met yet.")
            return None
        result = llm.complete(f"Finalize this result: {data} by formatting it in an easy-to-read format. Do not make up any information. Just use the information provided.")
        save_file_msg = await self.filesystem_agent.run(f"Write the final result to the a file called 'catering_result.txt': {result} and output a brief confirmation message")
        return StopEvent(result=str(result) + "\n" + str(save_file_msg))

In [14]:
workflow = CateringMultiAgentFlow(timeout=130, verbose=True)

guests1 = [
    {
        "is_vegan": True,
        "is_vegetarian": True,
        "is_gluten_free": False,
        "is_dairy_free": True,
        "allergens": []
    },
    {
        "is_vegan": True,
        "is_vegetarian": False,
        "is_gluten_free": False,
        "is_dairy_free": False,
        "allergens": ['quinoa','zucchini']   
    },
    {
        "is_vegan": False,
        "is_vegetarian": False,
        "is_gluten_free": False,
        "is_dairy_free": False,
        "allergens": ['zucchini'] # zucchini
    }
]

guest2 = [
  {
    "is_gluten_free": True,
    "allergens": ["nuts"]
  },
  {
    "is_gluten_free": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "allergens": ["chicken"]
  }
]  

guest3 = [
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "allergens": ["nuts"]
  },
  {
    "is_vegan": True,
    "is_vegetarian": True,
    "is_dairy_free": True,
    "allergens": ["chicken"]
  },
  {
    "allergens": ['egg', 'soy']
    }
]  

handler = workflow.run(
    guest_list=guest2,
    diet_analyze_agent=diet_analyze_agent,
    find_existing_recipe_agent=find_existing_recipes_agent,
    chef_matching_agent=chef_matching_agent,
    research_agent=research_agent,
    filesystem_agent=filesystem_agent,
    llm = llm
)

async for event in handler.stream_events():
    if isinstance(event, DietaryAnalysisEvent):
        print("===DietaryAnalysisEvent===")
        print(event.guest_list)
    elif isinstance(event, FindExistingRecipeEvent):
        print("===FindExistingRecipeEvent===")
        print(event.requirement)
    elif isinstance(event, SearchRecipeEvent):
        print("===SearchRecipeEvent===")
        print(event.requirement)
    elif isinstance(event, MatchChefEvent): 
        print("===MatchChefEvent===")
        print(event.recipe_list)
    elif isinstance(event, FinalizeEvent):
        print("===FinalizeEvent===")
        print(event.result)
final_result = await handler
print(final_result)

Running step setup
Step setup produced event DietaryAnalysisEvent
Running step diet_analyze
Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent
Requirements: [{'dietary_restrictions': ['is_vegetarian', 'is_gluten_free'], 'allergens': ['nuts', 'chicken']}]
Step diet_analyze produced no event
Running step find_existing_recipes
=== find_existing_recipes ===
find existing recipes result Faile